In [15]:
import MDAnalysis as mda
from MDAnalysis import transformations as trans

import numpy as np

In [2]:
# load trajectory
u = mda.Universe("../input_data/interface.psf",
                 "../input_data/trajectory.xyz")
dim = [16.869, 16.869, 41.478, 90, 90, 120]
transform = trans.boxdimensions.set_dimensions(dim)
u.trajectory.add_transformations(transform)
u.transfer_to_memory(step=100)

/home/jxzhu/.conda/envs/band-alignment/lib/python3.9/site-packages/MDAnalysis/coordinates/base.py:892: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")


# MDA v.s. m-MDA (serial version)

## MDA

In [3]:
from MDAnalysis.analysis.hydrogenbonds import HydrogenBondAnalysis

In [4]:
hbonds = HydrogenBondAnalysis(
    universe=u,
    donors_sel=None,
    hydrogens_sel="name H",
    acceptors_sel="name O",
    d_a_cutoff=3.0,
    d_h_a_angle_cutoff=150,
    update_selections=False
)
hbonds.run()
print(hbonds.results.hbonds.shape)
np.save("./mda_all.npy", hbonds.results.hbonds)

(31191, 6)


## m-MDA

In [5]:
from watdyn.m_mda import PartialHBAnalysis

In [6]:
hbonds = PartialHBAnalysis(
    universe=u,
    region=None,
    surf_ids=None,
    donors_sel=None,
    hydrogens_sel="name H",
    acceptors_sel="name O",
    d_a_cutoff=3.0,
    d_h_a_angle_cutoff=150,
    update_selections=False
)
hbonds.run()
print(hbonds.results.hbonds.shape)
np.save("./m-mda_all.npy", hbonds.results.hbonds)

(31191, 6)


## result check

In [7]:
data_mda = np.load("./mda_all.npy")
data_mmda = np.load("./m-mda_all.npy")
# check if two array are identical
mask = (data_mda - data_mmda) == 0
print(False in mask)
# if False, all element in mask is True

False


# serial v.s. parallel

In [8]:
from watdyn.m_mda import PartialHBAnalysis
from watdyn.parallel import parallel_exec

In [9]:
hbonds = PartialHBAnalysis(
    universe=u,
    region=None,
    surf_ids=None,
    donors_sel=None,
    hydrogens_sel="name H",
    acceptors_sel="name O",
    d_a_cutoff=3.0,
    d_h_a_angle_cutoff=150,
    update_selections=False
)
parallel_exec(hbonds.run,0,10000,1,4)
print(hbonds.results.hbonds.shape)
np.save("./m-mda_all_parallel.npy", hbonds.results.hbonds)

(31191, 6)


In [10]:
data_mmda = np.load("./m-mda_all.npy")
data_mmda_p = np.load("./m-mda_all_parallel.npy")
# check if two array are identical
mask = (data_mmda - data_mmda_p) == 0
print(False in mask)
# if False, all element in mask is True

False


# partial HB analysis

In [12]:
hbonds = PartialHBAnalysis(universe=u,
                           region=[2.7, 4.5],
                           surf_ids=[np.arange(871, 906),
                                     np.arange(691, 726)],
                           donors_sel=None,
                           hydrogens_sel="name H",
                           acceptors_sel="name O",
                           d_a_cutoff=3.0,
                           d_h_a_angle_cutoff=150,
                           update_selections=False)
hbonds.run()
print(hbonds.results.hbonds.shape)
np.save("./m-mda_interface_serial.npy", hbonds.results.hbonds)

[5.8477974, 35.679573]
[[ 8.54779739 10.34779739]
 [31.17957306 32.97957306]]
[5.832256, 35.68193]
[[ 8.53225584 10.33225584]
 [31.18193054 32.98193054]]
[5.846071, 35.67471]
[[ 8.54607077 10.34607077]
 [31.17470932 32.97470932]]
[5.821258, 35.678913]
[[ 8.52125807 10.32125807]
 [31.17891312 32.97891312]]
[5.808767, 35.672314]
[[ 8.50876684 10.30876684]
 [31.17231369 32.97231369]]
[5.7876444, 35.674698]
[[ 8.48764439 10.28764439]
 [31.17469788 32.97469788]]
[5.8218856, 35.690334]
[[ 8.52188559 10.32188559]
 [31.19033432 32.99033432]]
[5.8336, 35.69319]
[[ 8.53360004 10.33360004]
 [31.19319153 32.99319153]]
[5.8309364, 35.684826]
[[ 8.53093643 10.33093643]
 [31.1848259  32.9848259 ]]
[5.828984, 35.645935]
[[ 8.52898378 10.32898378]
 [31.14593506 32.94593506]]
[5.814769, 35.637512]
[[ 8.51476879 10.31476879]
 [31.13751221 32.93751221]]
[5.822207, 35.673]
[[ 8.52220697 10.32220697]
 [31.17300034 32.97300034]]
[5.8570814, 35.677876]
[[ 8.55708141 10.35708141]
 [31.17787552 32.97787552]]
[5

In [13]:
hbonds = PartialHBAnalysis(universe=u,
                           region=[2.7, 4.5],
                           surf_ids=[np.arange(871, 906),
                                     np.arange(691, 726)],
                           donors_sel=None,
                           hydrogens_sel="name H",
                           acceptors_sel="name O",
                           d_a_cutoff=3.0,
                           d_h_a_angle_cutoff=150,
                           update_selections=False)
parallel_exec(hbonds.run,0,10000,1,4)
print(hbonds.results.hbonds.shape)
np.save("./m-mda_interface_parallel.npy", hbonds.results.hbonds)

[5.8477974, 35.679573]
[[ 8.54779739 10.34779739]
 [31.17957306 32.97957306]]
[5.832256, 35.68193]
[[ 8.53225584 10.33225584]
 [31.18193054 32.98193054]]
[5.846071, 35.67471]
[[ 8.54607077 10.34607077]
 [31.17470932 32.97470932]]
[5.821258, 35.678913]
[[ 8.52125807 10.32125807]
 [31.17891312 32.97891312]]
[5.808767, 35.672314]
[[ 8.50876684 10.30876684]
 [31.17231369 32.97231369]]
[5.7876444, 35.674698]
[[ 8.48764439 10.28764439]
 [31.17469788 32.97469788]]
[5.8218856, 35.690334]
[[ 8.52188559 10.32188559]
 [31.19033432 32.99033432]]
[5.8336, 35.69319]
[[ 8.53360004 10.33360004]
 [31.19319153 32.99319153]]
[5.8309364, 35.684826]
[[ 8.53093643 10.33093643]
 [31.1848259  32.9848259 ]]
[5.828984, 35.645935]
[[ 8.52898378 10.32898378]
 [31.14593506 32.94593506]]
[5.814769, 35.637512]
[[ 8.51476879 10.31476879]
 [31.13751221 32.93751221]]
[5.822207, 35.673]
[[ 8.52220697 10.32220697]
 [31.17300034 32.97300034]]
[5.8570814, 35.677876]
[[ 8.55708141 10.35708141]
 [31.17787552 32.97787552]]
[5

In [14]:
data_s = np.load("./m-mda_interface_serial.npy")
data_p = np.load("./m-mda_interface_parallel.npy")
# check if two array are identical
mask = (data_s - data_p) == 0
print(False in mask)
# if False, all element in mask is True

False
